In [1]:
import pandas as pd
import datetime
import time
import requests
import numpy as np

In [10]:
def get_sudden_rising_df(idx):
    list_columns = [
        "종목명", "등락률", "현재가", "전일비", "시가", "고가", "저가", "거래량",
    ]
    list_int = [
        "현재가", "전일비", "시가", "고가", "저가", "거래량"
    ]
    
    url = f"https://finance.naver.com/sise/sise_low_up.naver?sosok={idx}"
    list_df = pd.read_html(url, encoding="euc-kr")

    df_sudden_rising = list_df[1]
    df_sudden_rising = df_sudden_rising[~df_sudden_rising["N"].isna()]
    
    for col in list_int:
        df_sudden_rising[col] = df_sudden_rising[col].astype(int)
    
    return df_sudden_rising[list_columns]

In [40]:
# 당일 급등한 종목의 최근 5일의 움직임을 알고 싶다
df_sudden_rising = pd.concat([get_sudden_rising_df(0), get_sudden_rising_df(1)])
df_sudden_rising = df_sudden_rising[["종목명", "등락률", "현재가", "전일비", "시가", "고가", "저가", "거래량"]]

list_nm = df_sudden_rising["종목명"].unique()

df_last_day = df_day_summary[(df_day_summary["DT"] == last_dt)]
df_target = df_last_day[df_last_day["NM"].isin(list_nm)]

df_target.head()

/var/folders/l2/31k7fcdd2xb41np3rpzw20lh0000gn/T/ipykernel_1105/611320729.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sudden_rising[col] = df_sudden_rising[col].astype(int)


,DT,CD,NM,FRGN_RT,END_PRC,VOL,PRC_AVG_5,VOL_AVG_5,PRC_AVG_10,VOL_AVG_10,PRC_MEDIAN,VOL_MEDIAN,REMAIN_SELL,REMAIN_BUY,REMAIN_BUY_AMOUNT,DAY_SUMMARY,WILL_BUY
1,2022-04-07,A095570,AJ네트웍스,4.99,6230,143976,6514,626600,6322,357232,6210,146736,7177,27673,49.0,"{'시가': '6330', '고가': '6360', '저가': '6200', '52...",20496
11,2022-04-07,A016380,KG동부제철,1.07,16350,28916095,12430,6102124,11805,3140158,11150,157586,88530,58544,-866.0,"{'시가': '16350', '고가': '16350', '저가': '14000', ...",-29986
21,2022-04-07,A010060,OCI,14.18,103500,209216,105500,160456,105550,229331,105250,190839,21295,47966,1689.0,"{'시가': '105000', '고가': '105500', '저가': '102500...",26671
26,2022-04-07,A009160,SIMPAC,7.73,8700,336361,8516,340032,8378,244189,8295,206961,7392,11713,19.0,"{'시가': '8650', '고가': '8830', '저가': '8490', '52...",4321
48,2022-04-07,A001260,남광토건,0.76,13450,41640,13720,30954,13845,60337,13875,40137,4913,9881,43.0,"{'시가': '13650', '고가': '13650', '저가': '13050', ...",4968


In [41]:
# 거래량 폭팔
df_day_summary = pd.read_csv("../bs_analysis/day_summary.csv", encoding="utf-8-sig")
df_day_summary["DT"].unique()
df_day_summary.columns

Index(['DT', 'CD', 'NM', 'FRGN_RT', 'END_PRC', 'VOL', 'PRC_AVG_5', 'VOL_AVG_5',
       'PRC_AVG_10', 'VOL_AVG_10', 'PRC_MEDIAN', 'VOL_MEDIAN', 'REMAIN_SELL',
       'REMAIN_BUY', 'REMAIN_BUY_AMOUNT', 'DAY_SUMMARY'],
      dtype='object')

In [42]:
list_columns = [
    "NM", "VOL", "VOL_MEDIAN", "END_PRC", "FRGN_RT", "PRC_MEDIAN", "WILL_BUY",
]
df_day_summary["WILL_BUY"] = df_day_summary["REMAIN_BUY"] - df_day_summary["REMAIN_SELL"]
last_dt = df_day_summary["DT"].max()
# 당일 거래량이 백만 이상이면서
df_preday = df_day_summary[(df_day_summary["DT"] == last_dt) & (df_day_summary["VOL"] > 2500000)][list_columns]
# 거래량이 15배 이상 증가한 경우
df_preday["VOL_RISING_RT"] = round(df_preday["VOL"] / df_preday["VOL_MEDIAN"], 2)
df_preday["PRC_RISING_RT"] = round(df_preday["END_PRC"] / df_preday["PRC_MEDIAN"], 2)
df_preday = df_preday[
    ((df_preday["PRC_RISING_RT"] < 1.1) & (df_preday["PRC_RISING_RT"] > 0.9)) &
    ((df_preday["VOL_RISING_RT"] < 50.0) & (df_preday["VOL_RISING_RT"] > 2.0))
]
df_preday = df_preday.sort_values(by=["VOL_RISING_RT"], ascending=False)

df_preday

,NM,VOL,VOL_MEDIAN,END_PRC,FRGN_RT,PRC_MEDIAN,WILL_BUY,VOL_RISING_RT,PRC_RISING_RT
345,케이티비네트워크,3336810,155456,4540,0.31,4495,46659,21.46,1.01
176,스틱인베스트먼트,5100419,392815,10050,6.63,10050,12640,12.98,1.00
174,스톤브릿지벤처스,5723250,467091,7610,0.42,7025,-2369,12.25,1.08
243,엠게임,10463960,863708,10750,6.72,10375,30815,12.12,1.04
191,아셈스,7341107,921256,15650,0.90,14500,13908,7.97,1.08
335,카카오,2871113,1424297,99900,28.99,105500,-37778,2.02,0.95


In [35]:
df_day_summary.columns

Index(['DT', 'CD', 'NM', 'FRGN_RT', 'END_PRC', 'VOL', 'PRC_AVG_5', 'VOL_AVG_5',
       'PRC_AVG_10', 'VOL_AVG_10', 'PRC_MEDIAN', 'VOL_MEDIAN', 'REMAIN_SELL',
       'REMAIN_BUY', 'REMAIN_BUY_AMOUNT', 'DAY_SUMMARY', 'WILL_BUY'],
      dtype='object')

/var/folders/l2/31k7fcdd2xb41np3rpzw20lh0000gn/T/ipykernel_1105/611320729.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sudden_rising[col] = df_sudden_rising[col].astype(int)


,DT,CD,NM,FRGN_RT,END_PRC,VOL,PRC_AVG_5,VOL_AVG_5,PRC_AVG_10,VOL_AVG_10,PRC_MEDIAN,VOL_MEDIAN,REMAIN_SELL,REMAIN_BUY,REMAIN_BUY_AMOUNT,DAY_SUMMARY,WILL_BUY
11,2022-04-07,A016380,KG동부제철,1.07,16350,28916095,12430,6102124,11805,3140158,11150,157586,88530,58544,-866.0,"{'시가': '16350', '고가': '16350', '저가': '14000', ...",-29986
18,2022-04-07,A001120,LX인터내셔널,25.72,33950,413440,34530,407034,34270,437476,34275,388061,8507,20153,542.0,"{'시가': '35350', '고가': '35550', '저가': '33950', ...",11646
49,2022-04-07,A008355,남선알미우,0.81,46100,110030,62480,123394,48890,100227,49050,108158,783,753,1.0,"{'시가': '57300', '고가': '57300', '저가': '45100', ...",-30
50,2022-04-07,A008350,남선알미늄,3.77,3365,24889435,3800,50430251,3726,85223046,3737,33345334,138376,595635,505.0,"{'시가': '3845', '고가': '3870', '저가': '3335', '52...",457259
78,2022-04-07,A145720,덴티움,17.30,72100,46916,72740,79026,72690,105144,72050,80389,6656,2160,-65.0,"{'시가': '71000', '고가': '72800', '저가': '70500', ...",-4496


In [7]:
# 행을 받아서 리스트로 던져줌
def remake_list_type_data(df_base, nm, col):
    list_result = []
    list_row = []
    for idx, row in df_base.iterrows():
        list_row.append(row[col])
    list_result.append([nm, list_row])
    
    return list_result

# 리스트 안에서 연속 증감
def serial_up_down_count(list_val):
    inc_cnt = 0
    dec_cnt = 0
    inc_seq = 0
    dec_seq = 0
    max_dec_seq = 0
    max_inc_seq = 0
    inc_tf = False
    dec_tf = False

    for p1, p2 in zip(list_val,list_val[1:]):
        if p2 > p1:
            inc_cnt += 1
            inc_seq += 1
            inc_tf = True
            dec_tf = False
        else:
            dec_cnt += 1
            dec_seq += 1
            inc_tf = False
            dec_tf = True
            
        if inc_tf:
            if dec_seq > max_dec_seq:
                max_dec_seq = dec_seq
            dec_seq = 0
        else:
            if inc_seq > max_inc_seq:
                max_inc_seq = inc_seq
            inc_seq = 0

    return [inc_cnt, max_inc_seq, dec_cnt, max_dec_seq]

In [8]:
# 당일 급등한 종목들의 어제까지의 추세를 알고싶다...
list_cols = [
    "VOL", "END_PRC", "FRGN_RT", "WILL_BUY"
]

# 당일 급등한 종목의 최근 5일의 움직임을 알고 싶다
df_sudden_rising = pd.concat([get_sudden_rising_df(0), get_sudden_rising_df(1)])
df_sudden_rising = df_sudden_rising[["종목명", "등락률", "현재가", "전일비", "시가", "고가", "저가", "거래량"]]

list_nm = df_sudden_rising["종목명"].unique()

for col in list_cols:
    list_df = []
    for nm in list_nm:
        df_nm = df_day_summary[df_day_summary["NM"] == nm][[col]].head(6)
        if df_nm.empty: continue
        list_trend = remake_list_type_data(df_nm, nm, col)
        list_df.append(pd.DataFrame(list_trend, columns=["NM", f"{col}_TREND".upper()]))
    globals()[f"df_{col}_trend".lower()] = pd.concat(list_df)

list_df_nm = []
for col in list_cols:
    list_df_nm.append(f"df_{col}_trend".lower())
    
for idx in range(len(list_df_nm)):
    if idx == 0: continue
    elif idx == 1:
        df_result = pd.merge(
            eval(list_df_nm[0]), eval(list_df_nm[1]), how="inner", on=["NM"]
        )
        continue
    df_result = pd.merge(
        df_result, eval(list_df_nm[idx]), how="inner", on=["NM"]
    )
        
df_result["VOL_UP_DN_RT"] = df_result["VOL_TREND"].apply(lambda X: serial_up_down_count(X))
df_result["END_PRC_UP_DN_RT"] = df_result["END_PRC_TREND"].apply(lambda X: serial_up_down_count(X))
df_result["FRGN_RT_UP_DN_RT"] = df_result["FRGN_RT_TREND"].apply(lambda X: serial_up_down_count(X))
df_result["WILL_BUY_UP_DN_RT"] = df_result["WILL_BUY_TREND"].apply(lambda X: serial_up_down_count(X))

df_result

/var/folders/l2/31k7fcdd2xb41np3rpzw20lh0000gn/T/ipykernel_1105/611320729.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sudden_rising[col] = df_sudden_rising[col].astype(int)


,NM,VOL_TREND,END_PRC_TREND,FRGN_RT_TREND,WILL_BUY_TREND,VOL_UP_DN_RT,END_PRC_UP_DN_RT,FRGN_RT_UP_DN_RT,WILL_BUY_UP_DN_RT
0,남선알미우,"[110030, 147759, 123549, 205976]","[46100, 57300, 75000, 59000]","[0.81, 0.81, 0.81, 0.81]","[-30, -183, -3825, 360]","[2, 1, 1, 1]","[2, 2, 1, 0]","[0, 0, 3, 0]","[1, 0, 2, 2]"
1,남선알미늄,"[24889435, 40393943, 155969193, 9496797, 14654...","[3365, 3745, 4100, 4060, 3730, 4320]","[3.77, 3.58, 4.07, 4.15, 4.15, 4.46]","[457259, 559367, 751083, 366134, 168465, 512761]","[4, 2, 1, 1]","[3, 2, 2, 2]","[3, 2, 2, 1]","[3, 2, 2, 2]"
2,현대비앤지스틸우,"[5761, 287, 289, 1556, 1488, 831]","[74700, 70200, 69700, 69700, 69500, 69800]","[0.59, 0.59, 0.59, 0.59, 0.59, 0.59]","[-38, 546, 312, 87, 369, 316]","[2, 2, 3, 1]","[1, 0, 4, 4]","[0, 0, 5, 0]","[2, 1, 3, 2]"
3,아남전자,"[457330, 730799, 2350796, 298425, 400407, 290475]","[2315, 2310, 2370, 2260, 2255, 2300]","[3.6, 3.82, 3.66, 3.6, 3.7, 3.72]","[-1626, 101354, -112195, -13783, 85002, 13376]","[3, 2, 2, 1]","[2, 1, 3, 2]","[3, 1, 2, 2]","[3, 2, 2, 1]"
4,KG동부제철,"[28916095, 837117, 150223, 110424, 155676, 156...","[16350, 12600, 11050, 11100, 11050, 11250]","[1.07, 0.98, 1.0, 1.0, 1.05, 1.06]","[-29986, 16955, -80926, -84076, -58665, -91516]","[2, 0, 3, 3]","[2, 1, 3, 2]","[3, 1, 2, 1]","[2, 1, 3, 2]"
5,덴티움,"[46916, 59907, 83059, 71879, 131089, 55357]","[72100, 72000, 72000, 73200, 74400, 72000]","[17.3, 17.07, 17.23, 17.36, 17.3, 17.26]","[-4496, 1578, 11987, 8908, -10334, 8039]","[3, 2, 2, 1]","[2, 2, 3, 2]","[2, 2, 3, 1]","[3, 2, 2, 2]"
6,테이팩스,"[45077, 31211, 20416, 16416, 15056, 26862]","[79700, 79600, 80800, 79500, 81000, 82000]","[1.47, 1.46, 1.46, 1.48, 1.56, 1.49]","[690, -955, -1569, -8032, 668, -575]","[1, 0, 4, 4]","[3, 1, 2, 1]","[2, 2, 3, 2]","[1, 1, 4, 3]"
7,코오롱글로벌,"[37799, 20058, 17431, 25111, 24400, 44612]","[21600, 22000, 22200, 22450, 22150, 22050]","[1.81, 1.83, 1.83, 1.83, 1.82, 1.82]","[4461, -1528, 2321, -3581, 2979, 10104]","[2, 1, 3, 2]","[3, 3, 2, 0]","[1, 1, 4, 0]","[3, 1, 2, 1]"
8,팬오션,"[4037385, 2428725, 4342048, 2622272, 3305531, ...","[6330, 6600, 6620, 6650, 6710, 6990]","[14.12, 13.99, 13.73, 13.57, 13.59, 13.64]","[-52082, 11008, -90701, 65913, 231966, -217179]","[2, 1, 3, 1]","[5, 0, 0, 0]","[2, 0, 3, 3]","[3, 2, 2, 1]"
9,LX인터내셔널,"[413440, 754533, 203514, 207194, 446866, 640987]","[33950, 35300, 34300, 34300, 34800, 34450]","[25.72, 25.51, 25.36, 25.38, 25.19, 25.05]","[11646, 3124, -2959, 11845, 19203, 9542]","[4, 1, 1, 1]","[2, 1, 3, 2]","[1, 1, 4, 2]","[2, 2, 3, 2]"


In [47]:
list_a = [560811, 199601, 276951, 274394, 220597]

for p1, p2 in zip(list_a,list_a[1:]):
    print(p1, p2, 1 if p1 < p2 else -1)

560811 199601 -1
199601 276951 1
276951 274394 -1
274394 220597 -1


In [39]:
# 가장 많이 사용된 문자의 반복 횟수
from itertools import groupby

def long_repeat(data):
    return max([len(list(g)) for k, g in groupby(data)], default=0)
               
long_repeat(list_a)

1

In [ ]:
for i in range(1, 11):
    globals()[f"my_variable_{i}"] = i

print()
print(my_variable_1)
print(my_variable_2)

In [ ]:
df_sudden_rising = pd.concat([get_sudden_rising_df(0), get_sudden_rising_df(1)])
df_sudden_rising["DT"] = dt = datetime.datetime.now().strftime("%Y-%m-%d")
df_sudden_rising = df_sudden_rising[["DT", "종목명", "등락률", "현재가", "전일비", "시가", "고가", "저가", "거래량"]]

df_pre = pd.read_csv("sudden_rising.csv", encoding="utf-8-sig")

df_result = pd.concat([df_sudden_rising, df_pre])

df_result.to_csv("sudden_rising.csv", encoding="utf-8-sig", index=False)

df_result["DT"].unique()

In [ ]:
# Load previous data
# df_sudden_rising = pd.read_csv("sudden_rising.csv", encoding="utf-8-sig")
df_sudden_rising

In [ ]:
list_ordered_key = ["CD","DT"]
df_day_summary.sort_values(by=list_ordered_key)[["NM", "DT", "FRGN_RT"]]

In [ ]:
# df_sudden_rising = df_sudden_rising.rename(columns={"종목명": "NM", "등락률": "UPDN_DESC"})
# df_sudden_rising.columns
last_dt = df_day_summary["DT"].max()
# lasta_dt = "2022-04-04"
df_day_summary = df_day_summary[df_day_summary["DT"] == last_dt]

df_base = pd.merge(
    df_sudden_rising, df_day_summary,
    how="inner",
    on=["NM"]
)

df_base["REMAIN_VOL"] = df_base["REMAIN_BUY"] - df_base["REMAIN_SELL"]
df_base["UPDN_RT"] = df_base["UPDN_DESC"].apply(lambda X: float(X.replace("%","")))
df_base[["NM", "UPDN_RT", "END_PRC", "PRC_MEDIAN", "VOL", "VOL_MEDIAN", "REMAIN_SELL", "REMAIN_BUY", "REMAIN_VOL", "REMAIN_BUY_AMOUNT"]]

In [ ]:
# 지난 거래일에 어떤 수치였을때 오늘 올랐을까?
# 최근 10일 거래량 중간값 비교
df_vol = df_base[["NM", "UPDN_RT", "VOL", "VOL_MEDIAN"]]
df_vol["VOL_RT"] = round(df_base["VOL"] / df_base["VOL_MEDIAN"], 2)
df_vol[(df_vol["VOL_RT"] > 10.0) & (df_vol["VOL"] > 2500000)].sort_values(by=["VOL_RT"], ascending=False)
# 최근 10일 종가 중간값 비교
# 매수대기
# 외국인 비율
# df_day_summary[df_day_summary["CD"] == "A078600"]

In [ ]:
df_day_summary

In [ ]:
# 처음 마지막 값
def make_first_median_value(df_base, col):
    df_first_last = df_base.groupby(["NM"]).agg(
        VOL=(col, "first"), 
        MEDIAN_VAL=("VOL_MEDIAN", "first")).reset_index()
    
    df_first_last["UPDN_RT"] = round((df_first_last["VOL"] / df_first_last["MEDIAN_VAL"]), 2)
    df_first_last = df_first_last[(df_first_last["UPDN_RT"] > 15.0) & (df_first_last["VOL"] > 2000000)]

    return df_first_last.sort_values(by=["UPDN_RT"], ascending=False).fillna(0.0)

df_vol = make_first_median_value(df_day_summary, "VOL")

In [ ]:
df_day_summary[df_day_summary["NM"] == "한국전자홀딩스"]

In [ ]:
df_frgn = df_day_summary[["DT","CD","NM","FRGN_RT"]]
df_frgn = pd.merge(
    df_vol, df_frgn,
    how="inner",
    on=["NM"]
)

df_frgn

In [ ]:
def check_frgn_rising(df_base):
    df_base = df_base.sort_values(by=["DT"])
    df_base["DIFF"] = df_base.FRGN_RT.diff()
    return df_base.tail(1)
    
list_nm = df_result["NM"].unique()
list_df = []
for nm in list_nm:
    list_df.append(check_frgn_rising(df_result[df_result["NM"] == nm].head(2)))
    
df_target = pd.concat(list_df)
df_target[df_target["DIFF"] > 0.0]

In [ ]:
dt = datetime.datetime.now().strftime("%Y-%m-%d")
dt